In [1]:
import pandas as pd
import numpy as np

In [2]:
akas = pd.read_csv('Data/title-akas-us-only.csv', low_memory=False)

In [3]:
basics = pd.read_csv('Data/data.tsv', sep='\t', low_memory=False)

In [4]:
filter_us_titles = basics['tconst'].isin(akas['titleId'])
basics = basics[filter_us_titles]
basics
df = basics
df.head()

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
0,tt0000001,short,Carmencita,Carmencita,0,1894,\N,1,"Documentary,Short"
1,tt0000002,short,Le clown et ses chiens,Le clown et ses chiens,0,1892,\N,5,"Animation,Short"
4,tt0000005,short,Blacksmith Scene,Blacksmith Scene,0,1893,\N,1,"Comedy,Short"
5,tt0000006,short,Chinese Opium Den,Chinese Opium Den,0,1894,\N,1,Short
6,tt0000007,short,Corbett and Courtney Before the Kinetograph,Corbett and Courtney Before the Kinetograph,0,1894,\N,1,"Short,Sport"


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1365643 entries, 0 to 10016966
Data columns (total 9 columns):
 #   Column          Non-Null Count    Dtype 
---  ------          --------------    ----- 
 0   tconst          1365643 non-null  object
 1   titleType       1365643 non-null  object
 2   primaryTitle    1365643 non-null  object
 3   originalTitle   1365643 non-null  object
 4   isAdult         1365643 non-null  object
 5   startYear       1365643 non-null  object
 6   endYear         1365643 non-null  object
 7   runtimeMinutes  1365643 non-null  object
 8   genres          1365643 non-null  object
dtypes: object(9)
memory usage: 104.2+ MB


In [6]:
# Convert values with placeholders back to null values
df = df.replace({'\\N':np.nan})

In [7]:
# Confirm changes
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1365643 entries, 0 to 10016966
Data columns (total 9 columns):
 #   Column          Non-Null Count    Dtype 
---  ------          --------------    ----- 
 0   tconst          1365643 non-null  object
 1   titleType       1365643 non-null  object
 2   primaryTitle    1365643 non-null  object
 3   originalTitle   1365643 non-null  object
 4   isAdult         1365643 non-null  object
 5   startYear       1266978 non-null  object
 6   endYear         37130 non-null    object
 7   runtimeMinutes  862524 non-null   object
 8   genres          1337027 non-null  object
dtypes: object(9)
memory usage: 104.2+ MB


In [8]:
# Convert startYear from object to float
df['startYear'] = df['startYear'].astype(float)

In [9]:
# Confirm changes
df['startYear'].dtype

dtype('float64')

In [10]:
# Create filter for films released after 2000
filter_begin_startYear = df['startYear'] >= 2000

In [11]:
# Create filter for films released in 2022 or prior to
filter_end_startYear = df['startYear'] <= 2022

In [12]:
# Combine to create filter for films released between 2000 and 2022
filter_year = filter_begin_startYear & filter_end_startYear
df = df[filter_year]

In [13]:
# Create a filter for titleType to include only movies
filter_titleType = df['titleType'].str.contains('movie')
df = df[filter_titleType]

In [14]:
# Create a filter for genres to exclude documentary films
filter_documentaries = df['genres'].str.contains('Documentary', na = False)
df = df[~filter_documentaries]

In [15]:
# Drop rows with null values in gneres and runtime
df = df.dropna(subset = ['genres', 'runtimeMinutes'])

In [16]:
# Confirm changes
df.info()
df.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 86979 entries, 34802 to 10016777
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   tconst          86979 non-null  object 
 1   titleType       86979 non-null  object 
 2   primaryTitle    86979 non-null  object 
 3   originalTitle   86979 non-null  object 
 4   isAdult         86979 non-null  object 
 5   startYear       86979 non-null  float64
 6   endYear         0 non-null      object 
 7   runtimeMinutes  86979 non-null  object 
 8   genres          86979 non-null  object 
dtypes: float64(1), object(8)
memory usage: 6.6+ MB


,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
34802,tt0035423,movie,Kate & Leopold,Kate & Leopold,0,2001.0,NaN,118,"Comedy,Fantasy,Romance"
61114,tt0062336,movie,The Tango of the Widower and Its Distorting Mi...,El tango del viudo y su espejo deformante,0,2020.0,NaN,70,Drama
67666,tt0069049,movie,The Other Side of the Wind,The Other Side of the Wind,0,2018.0,NaN,122,Drama
86793,tt0088751,movie,The Naked Monster,The Naked Monster,0,2005.0,NaN,100,"Comedy,Horror,Sci-Fi"
93930,tt0096056,movie,Crime and Punishment,Crime and Punishment,0,2002.0,NaN,126,Drama


In [17]:
# Save preprocessed data to csv
df.to_csv('Data/movie_basics')

In [18]:
# Load ratings file
ratings = pd.read_csv('Data/data-2.tsv', sep='\t', low_memory=False)

In [19]:
# filter ratings using title basics
filter_basics = ratings['tconst'].isin(basics['tconst'])
ratings = ratings[filter_basics]
df = ratings
df.head()
df.isna().sum()

tconst           0
averageRating    0
numVotes         0
dtype: int64

In [20]:
# Convert values with placeholders back to null values
df = df.replace({'\\N':np.nan})

In [21]:
# Confirm changes
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 504000 entries, 0 to 1331467
Data columns (total 3 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   tconst         504000 non-null  object 
 1   averageRating  504000 non-null  float64
 2   numVotes       504000 non-null  int64  
dtypes: float64(1), int64(1), object(1)
memory usage: 15.4+ MB


In [22]:
df.to_csv('Data/movie_ratings')